# Ceneo Scraper

# Ekstrakcja składowych pojedynczej opinii

|Składowa|Selektor|Zmienna|
|--------|--------|-------|
|id opinii|[data-entry-id]|opinion_id|
|autor|span.user-post__author-name|author|
|rekomendacja|span.user-post__author-recomendation|recommendation|
|gwiazdki|span.user-post__score-count|stars|
|treść|div.user-post__text|content|
|lista zalet|.review-feature__title ~ .review-feature__title--positives|pros|
|lista wad|  .review-feature__title ~ .review-feature__title--negatives |cons|
|dla ilu przydatna|button.vote-yes > span|helpful|
|dla ilu nieprzydatna| button.vote-no >span  |unhelpful|
|data wystawenia|span.user-post__published > time:nth-child(1)[datetime]|publish_date|
|data zakupu|span.user-post__published > time:nth-child(2)[datetime]|purchase_date|

## Załadowanie bibliotek

In [ ]:
import os
import json
import requests
from bs4 import BeautifulSoup

## Wysłanie zapytania do serwera

In [ ]:
product_id = "138331381"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews_scroll"


# Pobranie opinii z kodu HTML strony

## Pobranie wszystkich opinii o produkcie

In [ ]:

all_opinions = []
while(url):
    response = requests.get(url)
    page_dom = BeautifulSoup(response.text, "html.parser")
    opinions = page_dom.select("div.js_product-review")

    for opinion in opinions:
        single_opinion = {
        "opinion_id" : opinion["data-entry-id"],
        "author" : opinion.select_one(".user-post__author-name").text,
        "recomendation":opinion.select_one("span.user-post__author-recomendation > em").text.strip(),
        "rating":opinion.select_one("span.user-post__score-count").text.strip(),
        "content": opinion.select_one("div.user-post__text").text.strip(),
        "pros":[item.text.strip() for item in opinion.select("div.review-feature__title-positives ~ div.review-features__item")],
        "cons": [item.text.strip() for item in opinion.select("div.review-feature__title-negatives ~ div.review-features__item")],
        "useful": opinion.select_one("button.vote-yes > span").text.strip() ,
        "useless": opinion.select_one("button.vote-no > span").text.strip() ,
        "publish_date":  opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"],
        "purshare_date":  opinion.select_one("span.user-post__published > time:nth-child(2)")["datetime"]
    }
        all_opinions.append(single_opinion)
try:
    url = "https://www.ceneo.pl" + page_dom.select_one(a.pagination__next)["href"].strip()
except TypeError: url = None

In [ ]:
if not os.path.exists("opinions"):
        os.makedirs("opinions")
with open("f{opinions/product_id}.json", "w", encoding = "UTF-8") as jf:
    json.dump(all_opinions, jf  , indent=4, ensure_ascii =False )